# Spatial Dynamics: Markov based methods

Markov chain's assumption is that the observations are on the rows of the input and the different points in time on the columns.

In [1]:
import pysal
import numpy as np

# 3 states (a,b,c) and 5 different pixels at three different points in time.
# So the first pixel was in class ‘b’ in period 1, class ‘a’ in period 2, and class ‘c’ in period 3.
c = np.array([['b','a','c'],['c','c','a'],['c','b','c'],['a','a','b'],['a','b','c']])
c.shape, c

C:\Users\hyelim\Anaconda3\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


((5, 3), array([['b', 'a', 'c'],
        ['c', 'c', 'a'],
        ['c', 'b', 'c'],
        ['a', 'a', 'b'],
        ['a', 'b', 'c']], dtype='<U1'))

In [2]:
m = pysal.Markov(c)
m.classes  # array (k, 1), all different classes (bins) of the matrix.

array(['a', 'b', 'c'], dtype='<U1')

In [3]:
m.transitions  # matrix (k, k), count of transitions between each state i and j.

array([[1., 2., 1.],
       [1., 0., 2.],
       [1., 1., 1.]])

In [4]:
m.p  # matrix (k, k), transition probability matrix.

matrix([[0.25      , 0.5       , 0.25      ],
        [0.33333333, 0.        , 0.66666667],
        [0.33333333, 0.33333333, 0.33333333]])

In [5]:
m.steady_state  # matrix (k, 1), ergodic distribution.

matrix([[0.30769231],
        [0.28846154],
        [0.40384615]])

In [2]:
import geopandas as gpd
import pysal as ps
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
import datetime as dt
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import figure, scatter, show
from matplotlib import colors
import csv
import requests
import zipfile, urllib, os
from urllib.request import Request,urlopen, urlretrieve
import urllib
import io

import warnings
warnings.filterwarnings('ignore')

In [16]:
urls = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-{:02d}.csv'.format(a) for a in range(1,13)]

list_ = []
for url in urls:
    csv_response = requests.get(url)
    c = pd.read_csv(io.StringIO(csv_response.text)).sample(frac=0.001)
    list_.append(c)
    
df = pd.concat(list_)

In [48]:
df_clean = df[(df['passenger_count'] < 10) & 
                        (df['passenger_count'] > 0) & 
                        (df['extra'] >= 0) &
                        (df['extra'] <= 1) &
                        (df['RatecodeID'] < 7) &
                        (df['mta_tax'] >= 0) &
                        (df['trip_distance'] > 0) &
                        (df['tip_amount'] >= 0) &
                        (df['tolls_amount'] >= 0) &
                        (df['improvement_surcharge'] > 0) &
                        (df['total_amount'] > 0)]

df_clean = df_clean[(df_clean['payment_type'] == 1)] # only credit card payment

df_clean['tpep_pickup_datetime'] = df_clean.tpep_pickup_datetime.apply(
                                    lambda x:dt.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))


df_clean['pickup_month'] = df_clean.tpep_pickup_datetime.apply(lambda x: x.month)

df_clean = df_clean[(np.abs(stats.zscore(df_clean[['tip_amount']]))<2.5).all(axis=1)]

In [49]:
taxi_zone_tip_df = pd.pivot_table(df_clean, values='tip_amount', index=['PULocationID'], columns=['pickup_month'], aggfunc=np.mean)
taxi_zone_tip_df['PULocation'] = taxi_zone_tip_df.index
taxi_zone_tip_df.shape

(153, 13)

In [50]:
taxi_zones = gpd.read_file('taxi_zones_tip.shp')

In [51]:
taxi_zone_tip = pd.merge(taxi_zones, taxi_zone_tip_df, on='PULocation', how='left')

In [52]:
taxi_zone_tip.shape, taxi_zones.shape

((227, 25), (227, 13))

In [53]:
taxi_zone_tip.to_csv('taxi_zone_tip.csv', float_format='%.3f', index=False, header=True, sep=",", decimal=".")


## Classic Markov

In this section, all the spatial units are treated as independent
- The transition dynamics are assumed to hold for all units and for all time periods.
- Tnteractions between the transitions are ignored.

In [54]:
zone_tip = taxi_zone_tip[[1,2,3,4,5,6,7,8,9,10,11,12]]
#zone_tip = zone_tip.dropna()

zone_tip = np.array(zone_tip)
#zone_tip.index = taxi_zone_tip['zone']

In [55]:
zone_tip_q = np.array([ps.Quantiles(y).yb for y in zone_tip]).transpose()
zone_tip_q.shape

(12, 227)

In [56]:
markov_tip = ps.Markov(zone_tip_q)

In [57]:
markov_tip.classes

array([0, 1, 2, 3, 4])

In [58]:
markov_tip.transitions

array([[1507.,  118.,  107.,  108.,  133.],
       [ 109.,   25.,   16.,   17.,   17.],
       [ 111.,    7.,   11.,   19.,   20.],
       [ 102.,   16.,   12.,    7.,   25.],
       [ 135.,   20.,   24.,   13.,   33.]])

- count of transitions between each state i and j

In [59]:
markov_tip.p

matrix([[0.76381145, 0.0598074 , 0.05423213, 0.05473898, 0.06741004],
        [0.5923913 , 0.13586957, 0.08695652, 0.0923913 , 0.0923913 ],
        [0.66071429, 0.04166667, 0.06547619, 0.11309524, 0.11904762],
        [0.62962963, 0.09876543, 0.07407407, 0.04320988, 0.15432099],
        [0.6       , 0.08888889, 0.10666667, 0.05777778, 0.14666667]])

- transition probability matrix

In [60]:
markov_tip.steady_state

matrix([[0.72362234+0.j],
        [0.0687047 +0.j],
        [0.06280909+0.j],
        [0.06054931+0.j],
        [0.08431456+0.j]])

- ergodic distribution.
- Prob. of the 1st quantil state of tip amount = 72%



    * Ergodicity: a state i is said to be erdodic if it is aperiodic and positive recurrent.

In [61]:
ps.ergodic.fmpt(markov_tip.p)

matrix([[ 1.38193633, 15.78475864, 16.16508122, 16.38450127, 12.94027313],
        [ 1.66033   , 14.55504498, 15.54990309, 15.67922176, 12.51778618],
        [ 1.54774004, 16.01079371, 15.92126172, 15.42358438, 12.20478921],
        [ 1.60198185, 15.13934495, 15.74341272, 16.51546533, 11.79774135],
        [ 1.64676107, 15.29694245, 15.22368954, 16.25342777, 11.86034745]])

- calculates the matrix of first mean passage times for an ergodic transition probability matrix.

## Spatial Markov

The spatial Markov allows us to compare the global transition dynamics to those conditioned on regional context.


단순히 전체적으로 한 상태에서 다른 상태로 변환 간으한 확률값을 보여주는 것에서 더 나아가서 각 상태값을 갖는 지역들 가까이에 어던 상태의 지역이 있느냐에 따라 다음 상태로 넘어가는 확률을 보여준다.|

In [62]:
W = ps.queen_from_shapefile("taxi_zones_tip.shp")
W.transform = 'r'
W.sparse

('WARNING: ', 0, ' is an island (no neighbors)')
('WARNING: ', 94, ' is an island (no neighbors)')
('WARNING: ', 131, ' is an island (no neighbors)')
('WARNING: ', 170, ' is an island (no neighbors)')


<227x227 sparse matrix of type '<class 'numpy.float64'>'
	with 1118 stored elements in Compressed Sparse Row format>

In [63]:
f = pd.read_csv("taxi_zone_tip.csv",sep=',')
pci = np.array([f[str(y)] for y in [3,6,11]])

pci.shape

(3, 227)

In [64]:
pci = pci.transpose()

In [65]:
pci = np.nan_to_num(pci)
rpci = pci / (pci.mean(axis = 0))
rpci.shape


(227, 3)

In [66]:
sm = ps.Spatial_Markov(rpci, W, k = 5)

The global transition probability matrix for relative tip amount:

In [67]:
sm.p

matrix([[0.91954023, 0.01915709, 0.02298851, 0.03831418],
        [0.45454545, 0.27272727, 0.27272727, 0.        ],
        [0.04395604, 0.35164835, 0.49450549, 0.10989011],
        [0.15384615, 0.15384615, 0.40659341, 0.28571429]])

In [68]:
for p in sm.P:
    print(p)

[[0.94545455 0.01212121 0.01818182 0.02424242 0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.66666667 0.         0.         0.33333333 0.        ]
 [0.         0.         0.         0.         0.        ]]
[[1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.5  0.   0.25 0.25 0.  ]
 [0.   0.   0.   0.   0.  ]]
[[0.89830508 0.01694915 0.03389831 0.05084746 0.        ]
 [0.5        0.33333333 0.16666667 0.         0.        ]
 [0.33333333 0.33333333 0.11111111 0.22222222 0.        ]
 [0.5        0.1875     0.0625     0.25       0.        ]
 [0.         0.         0.         0.         0.        ]]
[[0.8        0.06666667 0.         0.13333333 0.        ]
 [0.         0.33333333 0.66666667 0.         0.        ]
 [0.02222222 0.42222222 0.48888889 0.06666667 0.        ]
 [0.03703704 0.18518519 0.40740741 0.37037037 0.        ]
 [0.         0.         0.         0.        

The different steady state distributions implied by these different transition probabilities:

- The prob. of the 1st quantil state remaining in the 1st quantil 
    - if their neighbors are in the 1st quantil: 0.945
    - if their neighbors are in the 5th quantil: 0.75

- The prob. of the 5th quantil state reamining in the 5th quantil: 0

In [42]:
for f in sm.F:
    print(f)

[[  1.06181779  78.66949632 158.36978515  30.50032936   1.        ]
 [  2.04997403  79.70028883 159.40057766  31.27632649   1.        ]
 [  2.04997403  79.70028883 159.40057766  31.27632649   1.        ]
 [  1.22690895  79.89640527 159.5966941   25.38179064   1.        ]
 [  2.04997403  79.70028883 159.40057766  31.27632649   1.        ]]
[[1.   1.   1.   1.5  1.  ]
 [2.   1.   1.   1.5  1.  ]
 [1.   1.   1.   1.5  1.  ]
 [1.75 1.   0.75 1.   1.  ]
 [2.   1.   1.   1.5  1.  ]]
[[ 1.47250032 31.09090909 16.10029499  7.24363636  1.        ]
 [ 2.07578254 31.06818182 14.33628319  5.97818182  1.        ]
 [ 3.32454695 27.45454545 11.4252704   3.60727273  1.        ]
 [ 2.64085667 28.81818182 13.09439528  4.97090909  1.        ]
 [ 2.889058   31.31469825 16.02963523  7.42742553  1.        ]]
[[ 1.         14.93333333  5.1         3.33333333  1.        ]
 [ 6.5         1.         10.2         6.66666667  1.        ]
 [ 6.5        14.78333333  1.          2.08333333  1.        ]
 [ 6.5       

- The 1st quantil state with neighbors in the 1st quantil
    - return to the 1st qauntil after 1.06 months after leaving the 1st quantil
    - they enter the 5th quanil a month after.

- The 1st quantil state within neighbors in the 5th quantil
    - return to the 1st quatil after 12.5 months
    - enter the 5th quantil after a month.

## LISA (Local Indicators of Spatial Associations) Markov

To consider the joint transitions of an observation and its spatial lag in the distribution
- the staties of the chain are defined as the four quadrants in the Moran scatter plot.

In [74]:
lm = ps.LISA_Markov(rpci, W)
lm.classes

array([1, 2, 3, 4])

In [75]:
lm.transitions

array([[151.,   9.,   7.,   6.],
       [  6.,  30.,  27.,   1.],
       [  6.,  15., 174.,   7.],
       [  2.,   2.,   6.,   5.]])

The estimated transition probability matrix is:

In [76]:
lm.p

matrix([[0.87283237, 0.05202312, 0.04046243, 0.03468208],
        [0.09375   , 0.46875   , 0.421875  , 0.015625  ],
        [0.02970297, 0.07425743, 0.86138614, 0.03465347],
        [0.13333333, 0.13333333, 0.4       , 0.33333333]])

- The diagonal values indicate the staying prob. and the staying prob. of the 1st and 3rd quantil is high.

The implied long run steady state distribution of the chain is:

In [77]:
lm.steady_state

matrix([[0.26779055],
        [0.11731473],
        [0.56865506],
        [0.04623967]])

- The 3rd quantil state might have a positive autocorrelation.

Finally the first mean passage time for the LISAs is:

In [78]:
ps.ergodic.fmpt(lm.p)

matrix([[ 3.73426178, 15.8043687 , 10.65911222, 30.63673994],
        [21.10261512,  8.52407885,  3.89309159, 31.72966029],
        [23.46986924, 13.84253481,  1.75853531, 30.77733371],
        [19.80244457, 12.96639462,  4.41044076, 21.62645352]])